In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9GAMMA_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (5000, 5000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 10000


In [5]:
from itertools import product
import numpy as np
from scipy.spatial.distance import cosine as cosine_distance
import gensim

Path to word2vec embeddings

In [6]:
word2vec_path = get_relative_path(prefix_path, DEAULT_WORDVECTORS)
os.path.exists(word2vec_path)

True

In [7]:
wordvectors = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

Load counts documents by word

In [8]:
docs_samples_word_doc_count_path = get_relative_path(data_path, DOCS_SAMPLES_WORD_DOC_COUNT)
samples_word_in_docs_count = load_pickle(docs_samples_word_doc_count_path)

Measure $idf(t, D) = log\frac{N}{\vert \left\{ d\in D:t\in d \right\} \vert}$

In [9]:
n_docs = len(docs_sample_a) + len(docs_sample_b)
t_idf = {t: np.log(n_docs/v) for t, v in samples_word_in_docs_count.items()}

In [10]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_content_for(index, samples_content, field='tokens'):
    return samples_content[index][field]

def tf_doc(terms):
    tf = Counter(terms)
    cnt = sum(tf.values())
    return {k: v/cnt for k, v in tf.items()}

m_measure_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    # Term frequencies by document
    tf_a = tf_doc(get_content_for(docs_sample_a[i], samples_content))
    tf_b = tf_doc(get_content_for(docs_sample_b[j], samples_content))
    # Term list
    t_list = list(set(tf_a) | set(tf_b))
    # TF-IDF
    tf_idf_a = np.zeros([len(t_list),])
    tf_idf_b = np.zeros([len(t_list),])
    # wordvector length 300
    centroid_a = mean = np.zeros(300, dtype=np.float64)
    centroid_b = mean = np.zeros(300, dtype=np.float64)
    for pos, t in enumerate(t_list):
        if t and t in wordvectors.vocab and t in t_idf:
            wv = wordvectors.get_vector(t)
            if t in tf_a:
                tf_idf_a[pos] = tf_a[t] * t_idf[t]
                centroid_a += wv * tf_idf_a[pos]
            if t in tf_b:
                tf_idf_b[pos] = tf_b[t] * t_idf[t]
                centroid_b += wv * tf_idf_b[pos]
            
    sum_tf_idf_a = sum(tf_idf_a)
    sum_tf_idf_b = sum(tf_idf_b)
    # Centroid
    centroid_a /= sum_tf_idf_a
    centroid_b /= sum_tf_idf_b
    
    # Measure
    measure_sim = 1.0 - cosine_distance(centroid_a, centroid_b)
    m_measure_sim[i, j] = measure_sim

print(m_measure_sim)

[[0.65026507 0.6352724  0.51125832 ... 0.70741633 0.53963897 0.63119034]
 [0.5617378  0.55335334 0.48273909 ... 0.61529872 0.60382952 0.46528783]
 [0.57255337 0.65461743 0.51519611 ... 0.66796163 0.57828285 0.62439063]
 ...
 [0.69814095 0.75335527 0.57345115 ... 0.66531861 0.70799736 0.64402167]
 [0.66652319 0.7462082  0.56672043 ... 0.72681021 0.63652852 0.61732316]
 [0.59589472 0.67401673 0.56727652 ... 0.68569787 0.6261404  0.58359683]]


In [11]:
del wordvectors

In [12]:
docs_samples_measure_sim_path = get_relative_path(data_path, DOCS_SAMPLES_WORD2VEC_TFIDF)
save_pickle(m_measure_sim, docs_samples_measure_sim_path)

In [13]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_measure_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_measure_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_measure_sim)

[[0.61990874 0.62639939 0.56540104 ... 0.64824639 0.58447769 0.61927054]
 [0.56964466 0.57560903 0.51955661 ... 0.59568461 0.53708647 0.56905821]
 [0.63615771 0.64281849 0.58022126 ... 0.66523814 0.59979795 0.63550278]
 ...
 [0.65475146 0.66160692 0.59718008 ... 0.68468186 0.61732897 0.65407738]
 [0.68577773 0.69295805 0.62547826 ... 0.71712642 0.64658193 0.68507171]
 [0.6174545  0.62391945 0.5631626  ... 0.64567996 0.58216373 0.61681882]]


In [14]:
docs_samples_measure_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_WORD2VEC_TFIDF_UDV)
save_pickle(m_udv_measure_sim, docs_samples_measure_sim_udv_path)